In [1]:
!pip install pyspark
!pip install findspark
!pip install plotly

     |████████████████████████████████| 281.3 MB 35 kB/s s eta 0:00:01 |                                | 61 kB 931 kB/s eta 0:05:02     |████████████▏                   | 107.4 MB 42.0 MB/s eta 0:00:05     |███████████████████▋            | 172.5 MB 53.6 MB/s eta 0:00:03     |████████████████████████▉       | 218.3 MB 57.6 MB/s eta 0:00:02
     |████████████████████████████████| 198 kB 65.8 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=270d79c88971ed6d4baa2e7092c4b44f7060bbad9de3187f0d28a240bf520b97
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, C

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import findspark
findspark.init

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px


from pyspark.sql import SparkSession # required to created a dataframe
spark=SparkSession.builder.appName("Basics").getOrCreate() 

from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from operator import add

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/08 09:58:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
file = '../input/bitcoin-tweets-20160101-to-20190329/tweets.csv'
schema = StructType([\
    StructField("id",DoubleType(),True), \
    StructField("user",StringType(),True), \
    StructField("fullname",StringType(),True), \
    StructField("url",StringType(),True), \
    StructField("timestamp",TimestampType(),True), \
    StructField("replies",IntegerType(),True), \
    StructField("likes",IntegerType(),True), \
    StructField("retweets",IntegerType(),True), \
    StructField("text",StringType(),True)])
df = spark.read.schema(schema).csv(file, sep=";", header=True,multiLine=True)

In [4]:
df.show(10)

+--------------------+---------------+-------------------+----+-------------------+-------+-----+--------+-------------------------------------+
|                  id|           user|           fullname| url|          timestamp|replies|likes|retweets|                                 text|
+--------------------+---------------+-------------------+----+-------------------+-------+-----+--------+-------------------------------------+
|1.132977055300300...|   KamdemAbdiel|      Abdiel kamdem|null|2019-05-27 11:49:14|      0|    0|       0|                 È appena uscito u...|
|1.132977073402736...|      bitcointe|          Bitcointe|null|2019-05-27 11:49:18|      0|    0|       0|                 Cardano: Digitize...|
|1.132977023893139...|      3eyedbran|Bran - 3 Eyed Raven|null|2019-05-27 11:49:06|      0|    2|       1|                 Another Test twee...|
|1.132977089089556...|  DetroitCrypto|        J. Scardina|null|2019-05-27 11:49:22|      0|    0|       0|                 Current

In [5]:
df.printSchema()

root
 |-- id: double (nullable = true)
 |-- user: string (nullable = true)
 |-- fullname: string (nullable = true)
 |-- url: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- replies: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- retweets: integer (nullable = true)
 |-- text: string (nullable = true)



In [6]:
file = '../input/bitcoin-tweets-16m-tweets-with-sentiment-tagged/mbsa.csv'
df2 = spark.read.option("escape","\"").csv(file, sep=",", header=True,multiLine=True)

In [7]:
df2.show()

+----------+-------------------------------------+---------+
|      Date|                                 text|Sentiment|
+----------+-------------------------------------+---------+
|2019-05-27|                 È appena uscito u...| Positive|
|2019-05-27|                 Cardano: Digitize...| Positive|
|2019-05-27|                 Another Test twee...| Positive|
|2019-05-27|                 Current Crypto Pr...| Positive|
|2019-05-27|                 Spiv (Nosar Baz):...| Positive|
|2019-05-27|                 #btc inceldiği ye...| Positive|
|2019-05-27|                 @nwoodfine We hav...| Positive|
|2019-05-27|                 @pedronauck como ...| Positive|
|2019-05-27|ブラジルはまぁ置いといてもドイツは...| Positive|
|2019-05-27|                 CHANGE IS COMING....| Positive|
|2019-05-27|                 One of the useful...| Positive|
|2019-05-27|                 Invested my Life ...| Positive|
|2019-05-27|                 Bitcoin Price Hit...| Positive|
|2019-05-27|  【毎日プレゼント企画】\n１万円分...| Posit

In [9]:
def func(val):
    if val == "Positive":
        return 1
    elif val == "Neutral":
        return 0
    return -1

func_udf = udf(func, IntegerType())
data_sam = df2.withColumn('score',func_udf(df2['Sentiment']))


sent = data_sam.select('Date','score').rdd.reduceByKey(add).toDF(['Date','score'])

In [10]:
sent.show()

+----------+------+
|      Date| score|
+----------+------+
|2019-05-27| 77949|
|2019-05-21|-82405|
|2019-05-22|-79836|
|2019-05-10| 49864|
|2019-05-26| 71683|
|2019-05-02|  1215|
|2019-05-25| 64414|
|2019-03-12|   958|
|2019-05-23| 73383|
|2019-05-24| 79386|
|2019-05-18|-63451|
|2018-09-11|  -941|
|2019-05-11| 93795|
|2019-05-20|-56668|
|2019-03-21|  -960|
|2019-05-03|  1367|
|2019-05-15| 91136|
|2019-05-13|110749|
|2019-03-09|   937|
|2018-12-29|  -844|
+----------+------+
only showing top 20 rows



In [15]:
scc = sent.select("score").sort('Date').rdd.flatMap(lambda x: x).collect()

In [17]:
min_date, max_date = sent.select(min("Date"), max("Date")).first()
min_date, max_date

('2014-09-18', '2019-11-23')

In [14]:
result_df = df.join(df2, ("text")).drop('Date')
result_df.show()

In [34]:
file = '../input/btcprice/BTC-USD.csv'
btc = spark.read.csv(file, sep=",", header=True,multiLine=True)
btc = btc.withColumn("Date", F.to_date(F.col("Date"),'yyyy-MM-dd').alias('dt'))

In [35]:
btc.show()

+----------+----------+----------+----------+----------+----------+--------+
|      Date|      Open|      High|       Low|     Close| Adj Close|  Volume|
+----------+----------+----------+----------+----------+----------+--------+
|2014-09-18|456.859985|456.859985|413.104004|424.440002|424.440002|34483200|
|2014-09-19|424.102997|427.834991|384.532013|394.795990|394.795990|37919700|
|2014-09-20|394.673004|423.295990|389.882996|408.903992|408.903992|36863600|
|2014-09-21|408.084991|412.425995|393.181000|398.821014|398.821014|26580100|
|2014-09-22|399.100006|406.915985|397.130005|402.152008|402.152008|24127600|
|2014-09-23|402.092010|441.557007|396.196991|435.790985|435.790985|45099500|
|2014-09-24|435.751007|436.112000|421.131989|423.204987|423.204987|30627700|
|2014-09-25|423.156006|423.519989|409.467987|411.574005|411.574005|26814400|
|2014-09-26|411.428986|414.937988|400.009003|404.424988|404.424988|21460800|
|2014-09-27|403.556000|406.622986|397.372009|399.519989|399.519989|15029300|

In [33]:
btc.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- Volume: string (nullable = true)



In [40]:
btc_d = btc.toPandas()

In [52]:
btc_d['Score'] = scc

In [53]:
btc_d.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Score
0,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200,-597
1,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700,-638
2,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600,643
3,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100,-582
4,2014-09-22,399.100006,406.915985,397.130005,402.152008,402.152008,24127600,678


## Price vs Score

In [72]:
Score = go.Bar(y=btc_d["Score"])
    
Price = go.Candlestick(x=btc_d.index, open=btc_d['Open'], high=btc_d['High'], low=btc_d['Low'], close=btc_d['Close'])

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(Score,)
fig.add_trace(Price,secondary_y=True)

fig.show()

## Year Wise Sentiment 

In [56]:
df3 = df2.withColumn('year', split(df2['Date'], '-').getItem(0))
df3.show()

+----------+-------------------------------------+---------+----+
|      Date|                                 text|Sentiment|year|
+----------+-------------------------------------+---------+----+
|2019-05-27|                 È appena uscito u...| Positive|2019|
|2019-05-27|                 Cardano: Digitize...| Positive|2019|
|2019-05-27|                 Another Test twee...| Positive|2019|
|2019-05-27|                 Current Crypto Pr...| Positive|2019|
|2019-05-27|                 Spiv (Nosar Baz):...| Positive|2019|
|2019-05-27|                 #btc inceldiği ye...| Positive|2019|
|2019-05-27|                 @nwoodfine We hav...| Positive|2019|
|2019-05-27|                 @pedronauck como ...| Positive|2019|
|2019-05-27|ブラジルはまぁ置いといてもドイツは...| Positive|2019|
|2019-05-27|                 CHANGE IS COMING....| Positive|2019|
|2019-05-27|                 One of the useful...| Positive|2019|
|2019-05-27|                 Invested my Life ...| Positive|2019|
|2019-05-27|               

In [57]:
df3.groupBy("Sentiment").pivot("year").count().show()

+---------+-----+-----+------+------+-------+-------+
|Sentiment| 2014| 2015|  2016|  2017|   2018|   2019|
+---------+-----+-----+------+------+-------+-------+
| Positive|44083|92835|264707|465872| 585783|7269932|
|  Neutral| null| null|  null|   400|   null|   null|
| Negative|46359|93026| 88417|481165|1578377|7441961|
+---------+-----+-----+------+------+-------+-------+



In [68]:
year_wise_sent = df3.groupBy("year").pivot("Sentiment").count().sort('year')
year_wise_sent.show()

+----+--------+-------+--------+
|year|Negative|Neutral|Positive|
+----+--------+-------+--------+
|2014|   46359|   null|   44083|
|2015|   93026|   null|   92835|
|2016|   88417|   null|  264707|
|2017|  481165|    400|  465872|
|2018| 1578377|   null|  585783|
|2019| 7441961|   null| 7269932|
+----+--------+-------+--------+



In [69]:
plott = year_wise_sent.toPandas()

In [70]:
fig = px.bar(plott, x="year", y=["Negative", "Neutral", "Positive"], title="Year-Wise Sentiment")
fig.show()